In [172]:
import numpy as np
import pandas as pd
import re
from scipy.stats import mode

In [173]:
train = pd.read_csv('~/datasets/titanic/train.csv')
test = pd.read_csv('~/datasets/titanic/test.csv')
y_train = train.pop('Survived')
train['source']='train'
test['source']='test'
df = pd.concat([train, test],ignore_index=True)
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,source
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train


In [174]:
df.apply(lambda x: sum(x.isnull()))

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
source            0
dtype: int64

In [175]:
df = df.drop(['PassengerId','Ticket'], axis=1)
df.tail(5)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,source
1304,3,"Spector, Mr. Woolf",male,NaN,0,0,8.0500,NaN,S,test
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,108.9000,C105,C,test
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,7.2500,NaN,S,test
1307,3,"Ware, Mr. Frederick",male,NaN,0,0,8.0500,NaN,S,test
1308,3,"Peter, Master. Michael J",male,NaN,1,1,22.3583,NaN,C,test


In [176]:
df.Age.fillna(df.Age.mean(), inplace = True) 
df.Fare.fillna(df.Fare.mean(), inplace = True)
df.Cabin.fillna('U', inplace = True)
df.Embarked.fillna(mode(df.Embarked).mode[0], inplace = True)
df.apply(lambda x: sum(x.isnull()))

/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
source      0
dtype: int64

In [177]:
df.tail(5)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,source
1304,3,"Spector, Mr. Woolf",male,29.881138,0,0,8.0500,U,S,test
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,108.9000,C105,C,test
1306,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,7.2500,U,S,test
1307,3,"Ware, Mr. Frederick",male,29.881138,0,0,8.0500,U,S,test
1308,3,"Peter, Master. Michael J",male,29.881138,1,1,22.3583,U,C,test


In [178]:
df['Deck'] = df['Cabin'].apply(lambda x: x[0])
df = df.drop('Cabin',axis=1)

In [179]:
df.Deck.value_counts()

U    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: Deck, dtype: int64

In [180]:
def AClass(x):
    if x < 19.0:
        return 1
    elif x >= 19.0 and x < 25.0:
        return 2
    elif x >= 25.0 and x < 29.881:
        return 3
    elif x >= 29.881 and x < 30.5:
        return 4
    elif x >= 30.4 and x < 40.1:
        return 5
    else:
        return 6

df['Age_Category'] = df['Age'].apply(lambda x: AClass(x)) 
df = df.drop('Age', axis=1)

In [181]:
def FClass(x):
    if x < 7.775:
        return 1
    elif x >= 7.775 and x < 8.662:
        return 2
    elif x >= 8.662 and x < 14.454:
        return 3
    elif x >= 14.454 and x < 26.0:
        return 4
    elif x >= 26.0 and x < 53.1:
        return 5
    else:
        return 6

df['Fare_Category'] = df['Fare'].apply(lambda x: FClass(x)) 
df = df.drop('Fare', axis=1)

In [182]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

In [183]:
def IAClass(x):
    if x > 1:
        return 0
    else:
        return 1

df['IsAlone'] = df['FamilySize'].apply(lambda x: IAClass(x)) 
df = df.drop(['SibSp','Parch','FamilySize'], axis=1)

In [184]:
df['Title'] = df['Name'].str.extract(r',\s(.*)\.', expand=False)
Title_Dictionary = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"

                    }


df['Title'] = df['Title'].apply(lambda x: df.Title.map(Title_Dictionary))
df = df.drop('Name', axis=1)

In [185]:
df.Sex = df.Sex.map({'male':1,'female':0})

In [186]:
df.tail(5)

,Pclass,Sex,Embarked,source,Deck,Age_Category,Fare_Category,IsAlone,Title
1304,3,1,S,test,U,4,2,1,Mr
1305,1,0,C,test,C,5,6,1,Mr
1306,3,1,S,test,U,5,1,1,Mr
1307,3,1,S,test,U,4,2,1,Mr
1308,3,1,C,test,U,4,4,0,Mr


In [187]:
categorical_columns = [x for x in df.dtypes.index if df.dtypes[x]=='object']
categorical_columns

['Embarked', 'source', 'Deck', 'Title']

In [188]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Embarked','Title','Deck']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
df = pd.get_dummies(df, columns=['Pclass','Embarked','Title','Deck','Age_Category','Fare_Category'])
df.head()

,Sex,source,IsAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_0,Embarked_1,Embarked_2,Title_0,...,Age_Category_3,Age_Category_4,Age_Category_5,Age_Category_6,Fare_Category_1,Fare_Category_2,Fare_Category_3,Fare_Category_4,Fare_Category_5,Fare_Category_6
0,1,train,0,0,0,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1,0,train,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2,0,train,1,0,0,1,0,0,1,1,...,1,0,0,0,0,1,0,0,0,0
3,0,train,0,1,0,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,1
4,1,train,1,0,0,1,0,0,1,1,...,0,0,1,0,0,1,0,0,0,0


In [189]:
train = df.loc[df['source']=="train"]
test = df.loc[df['source']=="test"]

train = train.drop('source',axis=1)
test = test.drop('source',axis=1)

X_train = train.loc[:,:]

X_test = test.loc[:,:]

In [190]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

model = xgb.XGBClassifier()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(X_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   15.1s finished


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.16, max_delta_step=0,
       max_depth=50, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1, verbose=1)

In [191]:
y_pred = model.predict(X_test)

In [192]:
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,

In [193]:
sample_file = pd.read_csv('~/datasets/titanic/gender_submission.csv')
sample_file['Survived'] = y_pred
sample_file.to_csv('titanic_submission2.csv', index=False)